# Chapter 8 - Exercises

### 1. Modify the depth-first search function to produce a topological sort.

In [2]:
from pythonds3.graphs import Graph

class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0

    def topological_sort(self):
        self.dfs()

        return sorted([vertex for vertex in self.get_vertices()], lambda vertex: vertex.get_closing_time(), reverse=True)

    def dfs(self):
        for vertex in self:
            vertex.color = "white"
            vertex.previous = -1
        for vertex in self:
            if vertex.color == "white":
                self.dfs_visit(vertex)

    def dfs_visit(self, start_vertex):
        start_vertex.color = "gray"
        self.time = self.time + 1
        start_vertex.discovery_time = self.time
        for next_vertex in start_vertex.get_neighbors():
            if next_vertex.color == "white":
                next_vertex.previous = start_vertex
                self.dfs_visit(next_vertex)
        start_vertex.color = "black"
        self.time = self.time + 1
        start_vertex.closing_time = self.time

### 2. Modify the depth-first search to produce strongly connected components.

We will need to compute the transpose of a graph with the following method.

In [9]:
from typing import Dict, List

def transpose(g: Dict[str, List]) -> Dict[str, List]:
    """
    Function used to transpose an input graph 'g'. 
    
    Input graph g is a dict where each key is the vertex v and each value
    an array of vertices w for which there is a directed edge
    from v to w.

    Example g = {
        "a": ["c", "d"],
        "b": ["d", "a"],
        "c": [],
        "d": ["c"]
    }
    """
    transposed_g = {v: [] for v in g.keys()}

    for v, adj_v in g.items():
        for w in adj_v:
            if v not in transposed_g[w]:
                transposed_g[w] += [v]

    return transposed_g

d1 = {"a": ["b", "c"], "b": ["c"], "c": []}

print(d1)
print(transpose({}))

{'a': ['b', 'c'], 'b': ['c'], 'c': []}
{}


In [ ]:
from pythonds3 import Stack

class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0
        self.components = []

    def dfs(self):
        for vertex in self:
            vertex.color = "white"
            vertex.previous = -1
        for vertex in self:
            if vertex.color == "white":
                self.dfs_visit(vertex)

    def transpose(self):
        transposed_g = DFSGraph()

        for vertex in self:
            for neighbor in vertex.get_neighbors():
                if neighbor.get_neighbor(vertex) is None:
                    transposed_g.add_edge(neighbor, vertex, vertex.get_neighbor(neighbor))

        return transposed_g


    def dfs_visit(self, start_vertex=None):
        if start_vertex is None:
            start_vertex = self.get_vertices()[0]
        
        start_vertex.color = "gray"
        self.time = self.time + 1
        start_vertex.discovery_time = self.time
        for next_vertex in start_vertex.get_neighbors():
            if next_vertex.color == "white":
                next_vertex.previous = start_vertex
                self.dfs_visit(next_vertex)
        start_vertex.color = "black"
        self.time = self.time + 1
        start_vertex.closing_time = self.time

    
    def dfs_scc_visit(self, start_vertex=None, scc_stack=None):
        if start_vertex is None:
            start_vertex = self.get_vertices()[0]

        if scc_stack is None:
            scc_stack = Stack()

        start_vertex.color = "gray"

        scc_stack.push(start_vertex.get_key())

        for next_vertex in sorted(start_vertex.get_neighbors(), key=lambda x: x.closing_time, reverse=True):
            if next_vertex.color == "white":
                next_vertex.previous = start_vertex
                self.dfs_scc_visit(next_vertex, scc_stack)

            if next_vertex.color == "gray" and next_vertex.get_key() in scc_stack:
                scc = []
                while not scc_stack.is_empty():
                    scc.append(scc_stack.pop())
                self.components += [scc]
        start_vertex.color = "black"

    
    def scc_algorithm(self, start_vertex=None):
        if start_vertex is None:
            start_vertex = self.get_vertices()[0]

        self.dfs_visit(start_vertex)
        transposed_g = self.transpose()
        transposed_g.dfs_scc_visit()

        